In [1]:
# Imports
import numpy as np
import uuid
from qsi.coordinator import Coordinator
from qsi.state import State, StateProp
from IPython.display import display

In [2]:
# Create coordinator and start the modules:
coordinator = Coordinator(port=20000)
# Coherent source
sps  = coordinator.register_component(module="single_photon_source.py", runtime="python")
mem = coordinator.register_component(module="memory_error.py", runtime="python")
# Run the coordinator process
coordinator.run()

In [3]:
# Initialize memory state
mem_state = mem.state_init()[0]

In [4]:
sp_state_prop = StateProp(
    state_type="light",
    truncation=2,
    wavelength=1550,
    polarization="R",
    uuid="first"
)
sp_state = State(sp_state_prop)

In [5]:
# Get the Kraus operators for the generation of the single photon state
response, operators = sps.channel_query(
    sp_state, {"input": sp_state_prop.uuid}
)
sp_state.apply_kraus_operators(operators, [sp_state_prop])

In [6]:
# Merging the memory state and the input optical state
whole_state = sp_state
whole_state.merge(mem_state)

In [7]:
response, operators = mem.channel_query(
    whole_state, {"input": sp_state_prop.uuid}
)
whole_state.apply_kraus_operators(operators, sp_state.get_all_props(response["kraus_state_indices"]))
print(whole_state.get_reduced_state([sp_state_prop]))
print(whole_state.get_reduced_state([mem_state.state_props[0]]))
print(response["error"])

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
0.0


In [8]:
# Creating the memory output state space
output_state_prop = StateProp(
    state_type = "light",
    truncation = 2,
    wavelength = 1550,
    polarization = "R",
    uuid = "second"
)
output_state = State(output_state_prop)

# Combining all of the states into a common (bigger) space
whole_state.merge(output_state)
print(whole_state.get_reduced_state([sp_state_prop]))
print(whole_state.get_reduced_state([mem_state.state_props[0]]))
print(whole_state.get_reduced_state([output_state_prop]))

response, operators = mem.channel_query(
    whole_state, {"input": output_state_prop.uuid}
)

print(response["kraus_state_indices"])
whole_state.apply_kraus_operators(operators, whole_state.get_all_props(response["kraus_state_indices"]))
print([p.uuid for p in whole_state.state_props])

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
['cc2301fe-7b1e-4cf1-87fc-7727259d5836', 'second']
['first', 'cc2301fe-7b1e-4cf1-87fc-7727259d5836', 'second']


In [9]:
print(whole_state.get_reduced_state(whole_state.get_all_props(["first"])))
print(whole_state.get_reduced_state([mem_state.state_props[0]]))
print(whole_state.get_reduced_state(whole_state.get_all_props(["second"])))

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
